# Preplanificación · Comparador por Identificación (Limpio & Modular)

Notebook conciso que **mantiene el funcionamiento** de tu código y lo organiza en módulos reutilizables.
- Carga & unifica todas las hojas de cada Excel (master por archivo).
- Normaliza encabezados (`snake_case`).
- Compara **`Cls-Dictadas`** por **`Identificación`** con alineación robusta y tolerancia (`EPS`).
- Reporta IDs donde **B < A** y permite depuración por ID.


## 1) Configuración

In [6]:
import pandas as pd
import re, unicodedata, os
from typing import Dict, List, Union

# Rutas de entrada (ajusta según tu entorno)
excel_a = r"C:\\Users\\andra\\Documents\\8vo\\Vice\\automatizacion\\ProyectoPreplanificacion\\Comparacion2025A\\data\\2025A.xlsx"
excel_b = r"C:\\Users\\andra\\Documents\\8vo\\Vice\\automatizacion\\ProyectoPreplanificacion\\Comparacion2025A\\data\\2025B.xlsx"

# Exportación
guardar_csv = True
salida_dir = "outputs_notebook"  

# Parámetros del dominio
IDENT_COL = "Identificación"
CLS_COL   = "Cls-Dictadas"


## 2) Utilidades de normalización

In [7]:
def remove_accents(s: str) -> str:
    """Elimina acentos/diacríticos de una cadena."""
    return "".join(ch for ch in unicodedata.normalize("NFKD", s) if not unicodedata.combining(ch))

def to_snake_case(s: str) -> str:
    """Convierte a snake_case: minúsculas, espacios/guiones -> '_', limpia símbolos."""
    s = re.sub(r"[^\w\s-]", " ", s, flags=re.UNICODE)
    s = re.sub(r"[-\s]+", "_", s.strip().lower())
    return re.sub(r"_+", "_", s)

def normalize_headers(df: pd.DataFrame) -> pd.DataFrame:
    """Normaliza todos los encabezados del DataFrame a snake_case."""
    out = df.copy()
    out.columns = [to_snake_case(remove_accents(str(c).strip())) or "columna" for c in df.columns]
    return out

def _normalize_number_string(s: str) -> str:
    """Adecúa coma/punto: '3,5'->'3.5', '1.234,56'->'1234.56'; quita NBSP/espacios."""
    s = ("" if s is None else str(s)).replace("\u00A0", "").replace(" ", "")
    s = re.sub(r"[^0-9,.-]", "", s)
    if s.count(",") > 0 and s.count(".") == 0:
        s = s.replace(",", ".")
    elif s.count(",") > 0 and s.count(".") > 0:
        s = s.replace(".", "").replace(",", ".")
    return s

def to_numeric(series: pd.Series) -> pd.Series:
    """Convierte Series a numérico robusto; no convertibles -> 0."""
    return pd.to_numeric(series.astype(str).map(_normalize_number_string), errors="coerce").fillna(0)

def normalize_identificacion_value(v: str) -> str:
    """Solo dígitos cuando hay >=8 dígitos; si no, alfanumérico sin símbolos."""
    if v is None:
        return ""
    s = str(v).strip().replace("\u00A0", "").replace(" ", "")
    s = remove_accents(s).lower()
    digits = re.sub(r"\D", "", s)
    return digits if len(digits) >= 8 else re.sub(r"[^0-9a-z]", "", s)

INVALID_TOKENS = {normalize_identificacion_value(t) for t in ["", "sin identificacion", "sin identificación", "s/i", "n/a", "na", "no aplica"]}

## 3) Carga de Excel y armado de `master`

In [8]:
def read_excel_all_sheets(path: str, sheets_cfg: Union[str, List[str]] = "all") -> Dict[str, pd.DataFrame]:
    """Lee todas las hojas del Excel como dataframes (dtype=object). Requiere openpyxl."""
    xls = pd.ExcelFile(path)
    sheets = xls.sheet_names if sheets_cfg == "all" else list(sheets_cfg)
    return {sh: pd.read_excel(path, sheet_name=sh, dtype="object") for sh in sheets}

def unify_sheets(frames: Dict[str, pd.DataFrame]) -> pd.DataFrame:
    """Normaliza encabezados y alinea columnas de todas las hojas, marcando el nombre de hoja en _sheet."""
    frames_norm = {sh: normalize_headers(df) for sh, df in frames.items()}
    all_cols = sorted(set().union(*[set(df.columns) for df in frames_norm.values()]))
    aligned = []
    for sh, df in frames_norm.items():
        tmp = df.copy()
        for c in all_cols:
            if c not in tmp.columns:
                tmp[c] = pd.NA
        tmp = tmp[all_cols]
        tmp["_sheet"] = sh
        aligned.append(tmp)
    out = pd.concat(aligned, ignore_index=True)
    out.insert(0, "row_index", out.index.astype(int))
    return out

def build_simple_dtypes(df: pd.DataFrame) -> Dict[str, str]:
    """Tipado simple por columna: int/float si numérico, si no string."""
    overrides = {}
    for c in df.columns:
        if c == "_sheet":
            overrides[c] = "string"; continue
        s = df[c]
        if s.dropna().empty:
            overrides[c] = "string"; continue
        nums = pd.to_numeric(s, errors="coerce")
        mask = s.notna()
        all_numeric = nums[mask].notna().all()
        overrides[c] = ("int" if (nums[mask] % 1 == 0).all() else "float") if all_numeric else "string"
    return overrides

def apply_dtypes(df: pd.DataFrame, dtypes: Dict[str, str]) -> pd.DataFrame:
    """Aplica dtypes inferidos con tolerancia a errores (fallback a string)."""
    out = df.copy()
    for col, t in dtypes.items():
        if col not in out.columns: continue
        try:
            if t == "string": out[col] = out[col].astype("string")
            elif t == "int": out[col] = pd.to_numeric(out[col], errors="coerce").astype("Int64")
            elif t == "float": out[col] = pd.to_numeric(out[col], errors="coerce")
            else: out[col] = out[col].astype("string")
        except Exception:
            out[col] = out[col].astype("string")
    return out

def prepare_master(excel_path: str) -> pd.DataFrame:
    """Pipeline: lee todas las hojas -> normaliza -> tipa -> row_index."""
    frames = read_excel_all_sheets(excel_path, "all")
    df = unify_sheets(frames)
    inferred = build_simple_dtypes(df)
    df = apply_dtypes(df, inferred)
    return df

master_a = prepare_master(excel_a)
master_b = prepare_master(excel_b)
print("A:", master_a.shape, "| B:", master_b.shape)
master_a.head(3)

c:\Users\andra\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\andra\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


A: (671, 69) | B: (654, 67)


,row_index,administracion_de_laboratorios_y_o_talleres_orientados_a_la_docencia,asesoria_tutoria_direccion_codireccion_y_o_calificacion_de_tesis_doctorales_y_tesis_de_maestria_de_investigacion,asesoria_tutoria_direccion_codireccion_y_o_calificacion_de_trabajos_de_titulacion_para_carreras_y_maestrias_profesionales,autoridades_academicas_decano_subdecanos_jefes_de_departamento_y_jefe_de_instituto_de_investigacion,cargo,cls_dictadas,coordinadores_de_carrera_y_de_programas_de_maestria,dedicacion,dedicacion_1,...,total_horas_actividades_de_docencia_fuera_del_1_1,total_horas_actividades_de_gestion,total_horas_actividades_de_investigacion,total_horas_d_i_g_p_v,total_horas_vacaciones_y_permisos,total_horas_vs_exigibles,uso_pedagogico_de_la_investigacion_y_la_sistematizacion_como_soporte_o_parte_de_la_ensenanza,vacaciones_2024_b,visitas_de_campo_tutorias_docencia_en_servicio_y_formacion_dual,_sheet
0,0,<NA>,<NA>,<NA>,<NA>,"PROFESOR PRINCIPAL A TIEMPO COMPLETO (NIVEL 1,...",162,<NA>,TC,TC,...,2,<NA>,586,992,80,0,<NA>,80,<NA>,Export
1,1,<NA>,152,<NA>,<NA>,"PROFESOR AGREGADO A TIEMPO COMPLETO (NIVEL 3, ...",172,<NA>,TC,TC,...,<NA>,95,527,992,80,0,<NA>,80,<NA>,Export
2,2,<NA>,<NA>,<NA>,<NA>,"PROFESOR AGREGADO A TIEMPO COMPLETO (NIVEL 3, ...",136,215,TC,TC,...,<NA>,215,440,992,80,0,<NA>,80,<NA>,Export


## 4) Comparación por Identificación (align + EPS)

In [9]:
def resolve_col(df: pd.DataFrame, raw_name: str) -> str:
    """Resuelve nombre humano a columna normalizada existente (match por substring más corto)."""
    target = to_snake_case(remove_accents(raw_name))
    if target in df.columns:
        return target
    candidates = [c for c in df.columns if target in c]
    if candidates:
        candidates.sort(key=len)
        return candidates[0]
    raise ValueError(f"No se encontró la columna '{raw_name}' (→ '{target}').")

def comparar_por_ident(master_a: pd.DataFrame, master_b: pd.DataFrame,
                       ident_col_h: str = "Identificación",
                       cls_col_h: str = "Cls-Dictadas",
                       eps: float = 1e-9):
    """Compara sumatorios de `Cls-Dictadas` por `Identificación` entre A y B y devuelve el reporte B < A."""
    id_a = resolve_col(master_a, ident_col_h)
    id_b = resolve_col(master_b, ident_col_h)
    cls_a = resolve_col(master_a, cls_col_h)
    cls_b = resolve_col(master_b, cls_col_h)

    A = master_a[[id_a, cls_a]].rename(columns={id_a: "ident_raw", cls_a: "cls"})
    B = master_b[[id_b, cls_b]].rename(columns={id_b: "ident_raw", cls_b: "cls"})

    A["ident"] = A["ident_raw"].map(normalize_identificacion_value)
    B["ident"] = B["ident_raw"].map(normalize_identificacion_value)

    A_valid = A.loc[~A["ident"].isin(INVALID_TOKENS)].copy()
    B_valid = B.loc[~B["ident"].isin(INVALID_TOKENS)].copy()

    A_valid["cls_num"] = to_numeric(A_valid["cls"])
    B_valid["cls_num"] = to_numeric(B_valid["cls"])

    a_series = A_valid.groupby("ident")["cls_num"].sum(); a_series.name = "cls_A"
    b_series = B_valid.groupby("ident")["cls_num"].sum(); b_series.name = "cls_B"

    a_aligned, b_aligned = a_series.align(b_series, join="inner")
    mask = (b_aligned + eps) < a_aligned

    reporte = pd.DataFrame({
        "identificacion_norm": a_aligned.index[mask],
        "cls_A": a_aligned[mask].astype(float).values,
        "cls_B": b_aligned[mask].astype(float).values,
    })
    reporte["diferencia_A_minus_B"] = reporte["cls_A"] - reporte["cls_B"]
    reporte = reporte.sort_values("diferencia_A_minus_B", ascending=False).reset_index(drop=True)

    resumen = {
        "ids_invalidas_en_A": int((A["ident"].isin(INVALID_TOKENS)).sum()),
        "ids_invalidas_en_B": int((B["ident"].isin(INVALID_TOKENS)).sum()),
        "ids_validas_A": int(a_series.size),
        "ids_validas_B": int(b_series.size),
        "ids_en_ambos": int(a_aligned.size),
        "casos_B_menor_que_A": int(len(reporte)),
    }
    cols_usadas = {"A": {"id": id_a, "cls": cls_a}, "B": {"id": id_b, "cls": cls_b}}
    return reporte, resumen, cols_usadas

# Ejecutar
reporte, resumen, cols = comparar_por_ident(master_a, master_b, IDENT_COL, CLS_COL)
print("Columnas usadas:", cols)
print(resumen)

Columnas usadas: {'A': {'id': 'identificacion', 'cls': 'cls_dictadas'}, 'B': {'id': 'identificacion', 'cls': 'cls_dictadas'}}
{'ids_invalidas_en_A': 11, 'ids_invalidas_en_B': 79, 'ids_validas_A': 640, 'ids_validas_B': 575, 'ids_en_ambos': 519, 'casos_B_menor_que_A': 183}


## 5) Exportar reporte

In [10]:
# ===== Reporte comercial (solo datos de B) =====
import os
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.formatting.rule import CellIsRule

# 1) Columnas pedidas (se deduplican conservando orden)
cols_h = [
    "Período Académico","Departamento","Tipo Cargo","Dedicación",
    "Profesor","Identificación","Profesor","Tipo","Cargo","Dedicación",
    "Fecha Inicio","Fecha Fin","Estado"
]
seen, cols_h_unique = set(), []
for h in cols_h:
    key = to_snake_case(remove_accents(h))
    if key not in seen:
        cols_h_unique.append(h); seen.add(key)

# 2) Resolver columnas en B y armar info base por Identificación (una fila por ID)
id_b = resolve_col(master_b, "Identificación")
resolved = {}
for h in cols_h_unique:
    try:
        resolved[h] = resolve_col(master_b, h)
    except Exception:
        # si alguna no existe, se omite silenciosamente
        pass

B_info = master_b[list({*resolved.values(), id_b})].copy()
B_info["identificacion_norm"] = B_info[id_b].map(normalize_identificacion_value)

# quedarnos con una fila por ID (la primera de B)
B_info = (B_info
          .dropna(subset=["identificacion_norm"])
          .sort_values(by=list(resolved.values()))
          .groupby("identificacion_norm", as_index=False)
          .first())

# 3) Unir con el reporte B<A (ya calculado) y ordenar por Departamento
df_final = B_info.merge(reporte, on="identificacion_norm", how="inner")

# colocar columnas pedidas al inicio (solo las que existan), luego métricas
ordered_front = [resolved[h] for h in cols_h_unique if h in resolved]
metric_cols   = ["cls_A","cls_B","diferencia_A_minus_B"]
other_cols    = [c for c in df_final.columns if c not in ordered_front + ["identificacion_norm"] + metric_cols]
df_final = df_final[ordered_front + ["identificacion_norm"] + metric_cols + other_cols]

# ordenar por Departamento si existe
try:
    dept_col = resolve_col(master_b, "Departamento")
    df_final = df_final.sort_values(by=dept_col, kind="stable")
except Exception:
    pass

# 4) Exportar Excel “comercial”
os.makedirs(salida_dir, exist_ok=True)
out_xlsx = os.path.join(salida_dir, "Reporte_Horas_Clase_B_menor_A.xlsx")
with pd.ExcelWriter(out_xlsx, engine="openpyxl") as xl:
    df_final.to_excel(xl, sheet_name="B_menor_A", index=False)

    wb = xl.book
    ws = wb["B_menor_A"]

    # encabezados: negrita + centro; autofiltro + freeze panes
    for cell in next(ws.iter_rows(min_row=1, max_row=1)):
        cell.font = Font(bold=True)
        cell.alignment = Alignment(horizontal="center")
    ws.auto_filter.ref = ws.dimensions
    ws.freeze_panes = "A2"

    # ancho de columnas
    for col in ws.columns:
        maxlen = max((len(str(c.value)) if c.value is not None else 0) for c in col)
        ws.column_dimensions[col[0].column_letter].width = min(max(12, maxlen + 2), 40)

    # formato numérico + resaltar Diferencia > 0
    headers = {c.value: i+1 for i, c in enumerate(ws[1])}
    for num_col in ["cls_A","cls_B","diferencia_A_minus_B"]:
        if num_col in headers:
            col_letter = ws.cell(row=1, column=headers[num_col]).column_letter
            for cell in ws[col_letter][1:]:
                cell.number_format = "#,##0.##"
    if "diferencia_A_minus_B" in headers:
        dcol = ws.cell(row=1, column=headers["diferencia_A_minus_B"]).column_letter
        ws.conditional_formatting.add(
            f"{dcol}2:{dcol}{ws.max_row}",
            CellIsRule(operator="greaterThan", formula=["0"],
                       fill=PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid"))
        )

print("Reporte horas clases generado:", out_xlsx)
display(df_final.head(20))


Reporte horas clases generado: outputs_notebook\Reporte_Horas_Clase_B_menor_A.xlsx


,periodo_academico,departamento,tipo_cargo,dedicacion,profesor,identificacion,tipo,cargo,fecha_inicio,fecha_fin,estado,identificacion_norm,cls_A,cls_B,diferencia_A_minus_B
12,2025-B,DEPARTAMENTO DE AUTOMATIZACION Y CONTROL INDUS...,TÉCNICO DOCENTE POLITÉCNICO,TC,IDENTIFICACIÓN: 0503631350/ NOMBRE: CHUQUI GAV...,0503631350,TÉCNICO DOCENTE POLITÉCNICO,TECNICO DOCENTE POLITECNICO A TIEMPO COMPLETO ...,2025-09-01 00:00:00,2026-02-28 00:00:00,INGRESADO,0503631350,396.0,324.0,72.0
18,2025-B,DEPARTAMENTO DE AUTOMATIZACION Y CONTROL INDUS...,TITULAR,TC,IDENTIFICACIÓN: 0604024158/ NOMBRE: TRUJILLO G...,0604024158,TITULAR,"PROFESOR AUXILIAR A TIEMPO COMPLETO (NIVEL 1, ...",2025-09-01 00:00:00,2026-02-28 00:00:00,INGRESADO,0604024158,276.0,234.0,42.0
46,2025-B,DEPARTAMENTO DE AUTOMATIZACION Y CONTROL INDUS...,TITULAR,TP,IDENTIFICACIÓN: 1705573630/ NOMBRE: SANCHEZ TE...,1705573630,TITULAR,"PROFESOR PRINCIPAL A TIEMPO PARCIAL (NIVEL 1, ...",<NA>,<NA>,INGRESADO,1705573630,144.0,0.0,144.0
48,2025-B,DEPARTAMENTO DE AUTOMATIZACION Y CONTROL INDUS...,TITULAR,TC,IDENTIFICACIÓN: 1705981981/ NOMBRE: RIVERA ARG...,1705981981,TITULAR,"PROFESOR PRINCIPAL A TIEMPO COMPLETO (NIVEL 1,...",2025-09-01 00:00:00,2026-02-28 00:00:00,INGRESADO,1705981981,216.0,180.0,36.0
78,2025-B,DEPARTAMENTO DE AUTOMATIZACION Y CONTROL INDUS...,TITULAR,TC,IDENTIFICACIÓN: 1712333721/ NOMBRE: GAMBOA BEN...,1712333721,TITULAR,"PROFESOR AUXILIAR A TIEMPO COMPLETO (NIVEL 1, ...",2025-09-01 00:00:00,2026-02-28 00:00:00,INGRESADO,1712333721,262.0,198.0,64.0
84,2025-B,DEPARTAMENTO DE AUTOMATIZACION Y CONTROL INDUS...,TITULAR,TC,IDENTIFICACIÓN: 1713199493/ NOMBRE: CRUZ DAVAL...,1713199493,TITULAR,"PROFESOR AGREGADO A TIEMPO COMPLETO (NIVEL 1, ...",2025-09-01 00:00:00,2026-02-28 00:00:00,INGRESADO,1713199493,180.0,126.0,54.0
87,2025-B,DEPARTAMENTO DE AUTOMATIZACION Y CONTROL INDUS...,TITULAR,TC,IDENTIFICACIÓN: 1713913752/ NOMBRE: CHAVEZ GAR...,1713913752,TITULAR,"PROFESOR AGREGADO A TIEMPO COMPLETO (NIVEL 2, ...",2025-09-01 00:00:00,2026-02-28 00:00:00,INGRESADO,1713913752,154.0,126.0,28.0
122,2025-B,DEPARTAMENTO DE AUTOMATIZACION Y CONTROL INDUS...,OCASIONAL,TC,IDENTIFICACIÓN: 1717557811/ NOMBRE: CHAMORRO H...,1717557811,OCASIONAL,PROFESOR OCASIONAL A TIEMPO COMPLETO - CONTRATO,2025-09-01 00:00:00,2026-02-28 00:00:00,INGRESADO,1717557811,248.0,234.0,14.0
140,2025-B,DEPARTAMENTO DE AUTOMATIZACION Y CONTROL INDUS...,TÉCNICO DOCENTE POLITÉCNICO,TC,IDENTIFICACIÓN: 1720037553/ NOMBRE: VEGA PINTO...,1720037553,TÉCNICO DOCENTE POLITÉCNICO,TECNICO DOCENTE POLITECNICO A TIEMPO COMPLETO ...,2025-09-01 00:00:00,2026-02-28 00:00:00,INGRESADO,1720037553,360.0,342.0,18.0
141,2025-B,DEPARTAMENTO DE AUTOMATIZACION Y CONTROL INDUS...,TÉCNICO DOCENTE POLITÉCNICO,TC,IDENTIFICACIÓN: 1720136637/ NOMBRE: DIAZ CAJAS...,1720136637,TÉCNICO DOCENTE POLITÉCNICO,TECNICO DOCENTE POLITECNICO A TIEMPO COMPLETO ...,2025-09-01 00:00:00,2026-02-28 00:00:00,INGRESADO,1720136637,360.0,324.0,36.0
